# **Tarea No. 4**

## Juan David Barrientos

### Planteamiento del Tabletime Scheduling

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#Definiciones:
#Curso: Es la definición de una materia que requieren los estudiantes. Por ejemplo Cálculo diferencial.
#Grupo: Es un espacio donde se asigna a un profesor un grupo de estudiantes. Para el curso anterior, puede haber muchos grupos, cada uno con un profesor diferente.
Ne=950  #Número de estudiantes
Nc=20   #Número de cursos disponibles.
Nh=20   #Número de horarios disponibles durante la semana. Cada bloque es de 1 hora.
Np=200  #Profesores que pudiéramos tener para dictar los cursos. No necesariamente se deben utilizar todos.
Nec=20  #Número máximo de estudiantes que puede haber en un grupo específico.
#Si muchos estudiantes requieren el mismo curso, entonces puede haber varios grupos del mismo curso. Sin embargo, por temas de horarios, no necesariamente
#si hay 100 estudiantes se pueden programar únicamente 5 grupos. A veces se pueden requerir más de 5.

#Matriz que indica qué cursos necesitan qué estudiantes. El valor de 1 indica que el estudiante de la fila i requiere el curso de la columna j
EstudiantesCursos=[]
for i in range(Ne):
  cursos=list(set(np.random.choice(a=range(Nc), replace=True, size=6)))
  EstudiantesCursos.append(cursos)

#Matriz que representa un profesor, qué cursos puede dictar. Un profesor no puede dictar cualquier tipo de clase. Solamente puede dictar los cursos que sean de su conocimiento
#Pero un profesor puede dictar varios grupos de un mismo curso. Con un límite de 4 cursos (si cada curso es de 4 horas, un profesor no puede dictar más de 16 horas a la semana).
ProfesoresCursos=[]
for i in range(Np):
  cursos=list(set(np.random.choice(a=range(Nc), replace=True, size=4)))
  ProfesoresCursos.append(cursos)


El problema que tenemos es lograr una asignación de programación que permita lograr la menor cantidad de profesores y que garantice que

a) Los estudiantes puedan tomar los cursos que necesitan en algún horario que no genere conflicto con otro cursos que ellos requieran.

b) Que los profesores tengan asignados cursos de los que ellos pueden dictar. El profesor no puede tener más de cuatro cursos asignados en horarios que no le generen conflicto.

c) Entonces es necesario generar un resultado de estudiantes, grupos (cursos), profesor y horario.

El punto más importante es lograr utilizar la menor cantidad de profesores o lograr los cursos con la mayor cantidad de usuarios cercanos a los 20 por curso.


In [ ]:
EstudiantesCursos[0] #Cursos que debe ver el primer estudiante

[1, 2, 7, 10, 13, 15]

In [ ]:
ProfesoresCursos[0] #Cursos que puede dar el primer profesor

[8, 16, 10, 13]

## Cromosoma

In [ ]:

Factor=15
def ftipo(i): #Esta Funcion se encarga de identificar la materia a la cual pertenece el curso
  return int(i/Factor)

def GeneradorPoblacion(Nc, Nh, Factor=15): # Se encarga de generar una matriz aleatorio de grupos y horarios
  Ng = Factor*Nc
  Mgh = []
  for i in range(Ng):
    dis = np.zeros(Nh)
    ale = np.random.randint(low=0,high=10)
    dis[ale] = 1
    Mgh.append(dis)
  return Mgh

def fitness(Nh, Nc, Nec, Mgh): #se encarga de evaluar los horarios encagando los estudiantes y profesores en los grupos
  NumeroEstudiantes=[0 for i in range(Factor*Nc)]
  NoAtendidos=0
  for i in range(len(EstudiantesCursos)):
    Hodisponibles=list(range(Nh))
    for ci in EstudiantesCursos[i]: #Todos los cursos que el estudiante requiere
      grupos=[]
      for grupo in range(len(Mgh)):
        if(ftipo(grupo)==ci): #El grupo corresponde al curso que el estudiante necesita.
          if(np.where(Mgh[grupo]==1)[0][0] in Hodisponibles and NumeroEstudiantes[grupo]<Nec): #Chequea los grupos que no estan llenos y no chocan con horarios del estudiante
            grupos.append(grupo)
      if(len(grupos)>0):
        eleccion=min(grupos) #se eligue el curso menos lleno
        NumeroEstudiantes[eleccion]+=1
        Hodisponibles.remove(np.where(Mgh[eleccion]==1)[0][0]) #se indica que ese horario ya no esta disponible
      else:
        NoAtendidos+=1 # si no hay cursos disponibles para el estudiante aumenta el número que no encontro grupo lo que genera que el fitness aumente
  NumeroProfesores=[0 for i in range(Factor*Nc)]
  ProfesoresUtilizados=[0 for i in range(len(ProfesoresCursos))]
  for i in range(len(ProfesoresCursos)):
    HopDisponibles=list(range(Nh))
    for ci in ProfesoresCursos[i]: #Todos los cursos que el profesor puede dar
      grupos=[]
      for grupo in range(len(Mgh)):
        if(ftipo(grupo)==ci): #El grupo corresponde al curso que el porfesor puede dar.
          if(np.where(Mgh[grupo]==1)[0][0] in HopDisponibles and NumeroProfesores[grupo]<1 and NumeroEstudiantes[grupo]>0 and len(HopDisponibles)>16): #se chequea que el profesor no choque horarios y que no se pase de 4 horarios de clase
            grupos.append(grupo)
      if(len(grupos)>0):
        eleccion=min(grupos)
        NumeroProfesores[eleccion]+=1
        HopDisponibles.remove(np.where(Mgh[eleccion]==1)[0][0]) #se indica que ese horario ya no esta disponible
        ProfesoresUtilizados[i]=1

  profesorestotales=np.sum(ProfesoresUtilizados)
  return (profesorestotales + 10*NoAtendidos), profesorestotales  #la funcion Fitness se basa en al numero de profesores y castiga el hecho de que estudiantes se hayan quedado sin cupo




## Algoritmo Genetico

In [ ]:
def prunning(poblacion, valorFitness,profesores):
  vmin=np.min(valorFitness)
  vmax=np.max(valorFitness)
  umbral=0.75*vmax+0.25*vmin #Se crea el umbrar basado en los valores maximos y minimos
  pob1=[]
  fit1=[]
  prof1=[]
  for i in range(len(poblacion)):
    if(valorFitness[i]<umbral): #Se eliminan las soluciones que no cumplen con el umbral
      pob1.append(poblacion[i])
      fit1.append(valorFitness[i])
      prof1.append(profesores[i])
  return pob1, fit1, prof1

def mutacion(poblacion, valorFitness,Profesores):
  pob2=[]
  fit2=[]
  prof2=[]
  for i in range(len(poblacion)):
    if(np.random.rand()<0.9):  #el 90% de las soluciones no muta
      pob2.append(poblacion[i])
      fit2.append(valorFitness[i])
      prof2.append(Profesores[i])
    else:
      mgh=poblacion[i]
      for j in range(len(mgh)):
        if(np.random.rand()<0.1):
          dis = np.zeros(Nh)
          ale = np.random.randint(low=0,high=10) # se le assigna un horario al azar
          dis[ale] = 1
          mgh[j]=dis
      fit,prof=fitness(Nh, Nc, Nec, mgh)
      pob2.append(mgh)
      fit2.append(fit)
      prof2.append(prof)
  return pob2, fit2, prof2

def crossOver(poblacion, valorFitness,profesores):
  posibles=[]
  pob3=[]
  fit3=[]
  prof3=[]
  vmin=np.min(valorFitness)
  vmax=np.max(valorFitness)
  umbral=0.75*vmax+0.25*vmin
  for i in range(len(poblacion)):
    pob3.append(poblacion[i])
    fit3.append(valorFitness[i])
    prof3.append(profesores[i])
    if(valorFitness[i]<umbral and np.random.rand()<0.5): # se separan las mejores soluciones para hacer el cross-over y el 50% de esas soluciones se cruzan
      posibles.append(poblacion[i])
  for k in range(10):
    pos1 = np.random.randint(low=0, high=len(posibles))
    pos2 = np.random.randint(low=0, high=len(posibles))
    while pos2 == pos1:
        pos2 = np.random.randint(low=0, high=len(posibles))
    suj1 = posibles[pos1]
    suj2 = posibles[pos2]
    suj3 = []
    cnt=0
    for i in range(len(suj1)):
      if(cnt<150):
        suj3.append(suj1[i])
      else:
        suj3.append(suj2[i])
      cnt+=1
    pob3.append(suj3)
    fitSuj3,prof=fitness(Nh, Nc, Nec, suj3)
    fit3.append(fitSuj3)
    prof3.append(prof)
  return pob3, fit3, prof3


In [ ]:
#Se crea una población inicial
Poblacion=[GeneradorPoblacion(Nc, Nh, Factor=15) for i in range(200)]
valores=[fitness(Nh, Nc, Nec, pi) for pi in Poblacion]
ValorFitness=[]
NumeroProfesores=[]
for i in range(len(valores)):
  ValorFitness.append(valores[i][0])
  NumeroProfesores.append(valores[i][1])

In [ ]:
Poblacion[0]

[array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 1., 

In [ ]:
ValorFitness[0]

149

In [ ]:
NumeroProfesores[0]

99

## Desarrollo

In [ ]:
mejor=[]
bestFitness=1e6
minimosprofes=0
from tqdm import tqdm
for i in tqdm(range(100)):
  Poblacion, ValorFitness, NumeroProfesores=prunning(Poblacion, ValorFitness,NumeroProfesores)
  Poblacion, ValorFitness, NumeroProfesores=mutacion(Poblacion, ValorFitness,NumeroProfesores)
  Poblacion, ValorFitness, NumeroProfesores=crossOver(Poblacion, ValorFitness,NumeroProfesores)
  mejor.append(np.min(ValorFitness))
  if(np.min(ValorFitness)<bestFitness):
    bestFitness=np.min(ValorFitness)
    ind=np.argmin(ValorFitness)
    bestPob=Poblacion[ind].copy()
    bestprof=NumeroProfesores[ind].copy()
    print(bestFitness)


  1%|          | 1/100 [00:30<49:45, 30.16s/it]

100


 14%|█▍        | 14/100 [07:08<48:38, 33.94s/it]

97


100%|██████████| 100/100 [1:10:45<00:00, 42.45s/it]


intente con mas iteraciones pero siempre se me caia el entorno de ejecucion

# **Solución**

In [ ]:
print(f"El mejor Fitness fue de: {bestFitness} con un numero de profesores de: {bestprof} haciendo 100 iteraciones")

El mejor Fitness fue de: 97 con un numero de profesores de: 97 haciendo 100 iteraciones


### **Este resultado deja a entender que ningun estudiante fue dejado por fuera y todos consiguen cupo**

In [ ]:
bestPob

[array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 